Топ водіїв за прибутком із поїздок, відстань яких перевищує 10 миль, за останній тиждень


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

spark = SparkSession.builder \
    .appName("SimpleApp") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/16 13:40:14 WARN Utils: Your hostname, DESKTOP-DP2FLCF resolves to a loopback address: 127.0.1.1; using 172.22.104.161 instead (on interface eth0)
25/04/16 13:40:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/16 13:40:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/16 13:40:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/16 13:40:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/04/16 13:40:17 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/04/16 13:40:17 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/04/16 13:40:17 WARN Utils: Service '

In [2]:
df_fare_init = spark.read.option("header", True).option("inferSchema", True).csv("../data_lake/trip_fare_1.csv")
df_trip_init = spark.read.option("header", True).option("inferSchema", True).csv("../data_lake/trip_data_1.csv")

In [3]:
for column in df_fare_init.columns:
    df_fare_init = df_fare_init.withColumnRenamed(column, column.strip().replace(" ", "_"))

In [4]:
from pyspark.sql.functions import to_date, lit

df_trip = df_trip_init.filter(
    (col("passenger_count") > 0) & (col("passenger_count") <= 10) &
    (col("trip_time_in_secs") > 0) &
    (col("trip_distance") > 0) &
    (col("pickup_latitude").between(40.5, 41.0)) &
    (col("dropoff_latitude").between(40.5, 41.0)) &
    (col("pickup_longitude").between(-74.5, -73.0)) &
    (col("dropoff_longitude").between(-74.5, -73.0)) &
    (col("pickup_datetime") <= lit("2013-01-27"))
)

df_fare = df_fare_init.filter(
    (col("fare_amount") >= 0) &
    (col("tip_amount") >= 0) &
    (col("tolls_amount") >= 0) &
    (col("total_amount") >= 0) &
    ((col("fare_amount") > 0) | (col("payment_type") == "NOC")) &
    (col("pickup_datetime") <= lit("2013-01-27"))
)

In [5]:
from pyspark.sql.functions import max, min, col, to_date, trunc

df = df_trip.join(df_fare, on=["medallion", "hack_license", "pickup_datetime", "vendor_id"])


In [6]:
from pyspark.sql.functions import col, to_date, date_sub, lit, sum as Fsum, round as Fround, count


In [7]:
week_start = date_sub(lit("2013-01-27"), 6)

In [8]:
df_last_week = df.withColumn("trip_date", to_date("pickup_datetime")) \
    .filter((col("trip_date") >= week_start) & (col("trip_distance") > 10))

df_top_drivers = df_last_week.groupBy("hack_license").agg(
    Fround(Fsum("total_amount"), 2).alias("total_earnings"),
    count("*").alias("long_trip_count")
).orderBy(col("total_earnings").desc())


In [9]:
df_top_drivers.show(20, truncate=False)

+--------------------------------+--------------+---------------+
|hack_license                    |total_earnings|long_trip_count|
+--------------------------------+--------------+---------------+
|3895ECEDAF11644ADDDCCFB57F10A205|1730.7        |31             |
|4C41D9F8071DC7FA9A526ADACD6DAD77|1695.15       |31             |
|D379EFF4BC03AD70DAB2729A1A0CFF21|1658.64       |31             |
|D92C6162E7E82E115E75A8CFDD435FE3|1604.4        |27             |
|6B5C69870E8775E65570B7CF0B4B41CC|1600.35       |28             |
|5114DF85775775ED4F53235D8478E80B|1594.85       |28             |
|5682C07954E9B555F3809EB59CA3EA51|1579.95       |28             |
|E77D8148C5C74128D72F3FEEA85E93F1|1575.62       |23             |
|6EC4BD844C40B9A941D4F67DC926E453|1574.7        |28             |
|6CD44E75DB4B2E74D44717F6C7FB566C|1555.22       |27             |
|59EE2CA458BC2BA9A5F1452E5531F06A|1497.43       |25             |
|76801FA0D6F094C97FE8716AB64FFF41|1495.65       |27             |
|8FC3DA135

In [ ]:
df_top_drivers.coalesce(1).write.mode("overwrite").option("header", True).csv("./results/top_drivers_by_earnings.csv")